In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [98]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [99]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [100]:
X=df.drop(columns=['Outcome'])
y=df['Outcome']

In [101]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [102]:
X.shape

(768, 8)

In [103]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [104]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [105]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [106]:
history=model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6425 - loss: 0.6514 - val_accuracy: 0.7100 - val_loss: 0.5945
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6853 - loss: 0.6114 - val_accuracy: 0.7273 - val_loss: 0.5634
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7020 - loss: 0.5805 - val_accuracy: 0.7273 - val_loss: 0.5393
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7244 - loss: 0.5559 - val_accuracy: 0.7446 - val_loss: 0.5205
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7467 - loss: 0.5358 - val_accuracy: 0.7446 - val_loss: 0.5065
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7598 - loss: 0.5199 - val_accuracy: 0.7619 - val_loss: 0.4964
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7598 - loss: 0.5078 - val_accuracy: 0.7576 - val_loss: 0.4875
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7709 - loss: 0.4971 - val_accuracy: 0.7489 - val_loss: 0.4814
Epo

In [107]:
#1. how to select the best optimizer
#2. No of nodes in a layer
#3. how to select no of layers
#4. All in all model
import kerastuner as kt

In [108]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgp','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [109]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5)


In [110]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.7748917937278748
Total elapsed time: 00h 00m 05s


In [111]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [112]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [113]:
model.fit(X_train,y_train,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

In [114]:
#to find the number of the neurons
def build_model(hp):
    model=Sequential()
    units=hp.Int('units', min_value=8, max_value=128, step=6)
    model.add(Dense(units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [115]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [116]:
tuner.search(X_train,y_train,
             epochs=5,
             validation_data=(X_test,y_test))


In [117]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [118]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(8, 8), Received: value.shape=(8, 32). Target variable: <Variable path=sequential/dense/kernel, shape=(8, 8), dtype=float32, value=[[-0.39013255 -0.5275947   0.5623724  -0.561304   -0.47676867 -0.1456627
   0.09947789 -0.2019262 ]
 [ 0.4948321  -0.40683156 -0.15012112 -0.10241342  0.13247615  0.17419624
  -0.4572352   0.2951998 ]
 [-0.5236236   0.15778983  0.20028007 -0.56821007 -0.46114916  0.28661394
   0.33991277 -0.26760826]
 [-0.42694014  0.30097193  0.40230697 -0.03878635 -0.15084645  0.1748575
   0.32604516  0.13517076]
 [ 0.26201844 -0.5352267  -0.24770978 -0.00305986  0.43560773 -0.5966734
   0.08449644  0.2967636 ]
 [ 0.46317977 -0.44777593  0.4127862   0.10169554 -0.00263095  0.5402579
   0.5499398  -0.40430078]
 [-0.40498084  0.45290023

In [ ]:
model.fit(X_train,y_train,initial_epoch=100,epochs=100,verbose=1)

In [120]:
#to find the number of the layers
def build_model(hp):
    model=Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [121]:
tuner=kt.RandomSearch(
    build_model,
    max_trials=5,
    directory='asmit',
    project_name='asmit'
    ,objective='val_loss'
)

In [122]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_loss: 0.529908299446106

Best val_loss So Far: 0.5187355875968933
Total elapsed time: 00h 00m 06s


In [123]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [124]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [125]:
model.fit(X_train,y_train,initial_epoch=6,epochs=100,validation_data=(X_test,y_test))

Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7561 - loss: 0.5012 - val_accuracy: 0.7835 - val_loss: 0.4738
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7858 - loss: 0.4614 - val_accuracy: 0.7749 - val_loss: 0.4693
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7896 - loss: 0.4470 - val_accuracy: 0.7706 - val_loss: 0.4682
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8082 - loss: 0.4312 - val_accuracy: 0.7619 - val_loss: 0.4793
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7970 - loss: 0.4255 - val_accuracy: 0.7619 - val_loss: 0.4922
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8231 - loss: 0.4149 - val_accuracy: 0.7619 - val_loss: 0.4840
Epoch 13/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8250 - loss: 0.4013 - val_accuracy: 0.7576 - val_loss: 0.5006
Epoch 14/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8268 - loss: 0.3939 - val_accuracy: 0.74

In [131]:
#to find the number of the layers
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(Dense(
                hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                activation=hp.Choice('actiavtion',values=['relu','sigmoid','tanh']),
                input_dim=8))
        else:
            model.add(Dense(
                hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                activation=hp.Choice('actiavtion',values=['relu','sigmoid','tanh'])))
        counter+=1
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [132]:
tuner=kt.RandomSearch(
    build_model,
    max_trials=3,
    directory='asmit',
    project_name='asmit'
    ,objective='val_loss'
)

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [133]:
tuner.search(X_train,y_train,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_loss: 0.586138129234314

Best val_loss So Far: 0.4788186252117157
Total elapsed time: 00h 00m 04s


In [134]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 48,
 'actiavtion': 'tanh',
 'units1': 72,
 'units2': 24,
 'units3': 88,
 'units4': 48,
 'units5': 48}

In [135]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [136]:
model.fit(X_train,y_train,initial_epoch=6,epochs=100,validation_data=(X_test,y_test))

Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7672 - loss: 0.4825 - val_accuracy: 0.7359 - val_loss: 0.4834
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7970 - loss: 0.4605 - val_accuracy: 0.7359 - val_loss: 0.4958
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7896 - loss: 0.4618 - val_accuracy: 0.7532 - val_loss: 0.5050
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7877 - loss: 0.4614 - val_accuracy: 0.7403 - val_loss: 0.4866
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7989 - loss: 0.4547 - val_accuracy: 0.7489 - val_loss: 0.4878
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7877 - loss: 0.4516 - val_accuracy: 0.7403 - val_loss: 0.4944
Epoch 13/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7896 - loss: 0.4490 - val_accuracy: 0.7186 - val_loss: 0.4919
Epoch 14/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7877 - loss: 0.4398 - val_accuracy: 0.75